In [1]:
import random
import networkx as nx
import itertools

In [2]:
def load_graph(file_path):
    G = nx.Graph() 

    with open(file_path, "r") as file:
        for line in file:
            parts = line.split()
            
            node1 = int(parts[0]) 
            node2 = int(parts[1])  
            G.add_edge(node1, node2) 
    return G

In [3]:
def load_gcol(file_path):
    G = nx.Graph() 

    with open(file_path, "r") as file:
        for line in file:
            parts = line.split()
            if parts[0] == 'e':
                node1 = int(parts[1]) 
                node2 = int(parts[2])  
                G.add_edge(node1, node2) 
    return G

In [4]:
def is_resolving_set(B, G):
    distances = {}
    for v in G.nodes:
        distances[v] = [nx.shortest_path_length(G, v, u) for u in B]
    
    for v1, v2 in itertools.combinations(G.nodes, 2):
        if distances[v1] == distances[v2]:
            return False
    return True

In [5]:
def fitness(resolving_set, G,shortest_paths):
    unresolved_pairs = 0
    node_list = list(G.nodes)  # Pravi čvorovi iz grafa

    resolving_set = {node_list[i] for i in resolving_set if i < len(node_list)}  # Sigurno mapiranje

    for u, v in itertools.combinations(G.nodes, 2):
        if not any(abs(shortest_paths[u][j] - shortest_paths[v][j]) > 0 for j in resolving_set if j in shortest_paths[u]):
            unresolved_pairs += 1  # Par (u, v) nije razrešen

    return len(resolving_set) + 100 * unresolved_pairs  # Penalizacija nerešenih parova


In [6]:
def initialize_particles(num_particles, n):
    particles = []
    velocities = []
    
    for _ in range(num_particles):
        particle = [random.randint(0, 1) for _ in range(n)]  # Binarni niz
        velocity = [random.uniform(-1, 1) for _ in range(n)]  # Brzina svake komponente
        particles.append(particle)
        velocities.append(velocity)
    
    return particles, velocities

In [7]:
def update_particle(particle, velocity, pBest, gBest, w, c1, c2):
    new_velocity = []
    new_particle = []

    for i in range(len(particle)):
        r1 = random.random()
        r2 = random.random()

        # Ažuriranje brzine
        v_new = w * velocity[i] + c1 * r1 * (pBest[i] - particle[i]) + c2 * r2 * (gBest[i] - particle[i])
        new_velocity.append(v_new)

        # Sigmoidna aktivacija za binarni PSO
        prob = 1 / (1 + pow(2.718, -v_new))  # Sigmoidna funkcija
        new_particle.append(1 if random.random() < prob else 0)  # Konvertovanje u 0 ili 1
    
    return new_particle, new_velocity

In [8]:
def particle_swarm_optimization(G, num_particles, num_iterations, w, c1, c2,shortest_paths):
    node_list = list(G.nodes) 

    particles, velocities = initialize_particles(num_particles, len(node_list))
    pBest = particles[:]  # Najbolje pojedinačne pozicije
    pBest_fitness = [fitness({i for i, bit in enumerate(p) if bit == 1 and i < len(node_list)}, G,shortest_paths) for p in particles]

    # Pronalazak početnog gBest
    gBest = particles[pBest_fitness.index(min(pBest_fitness))]
    gBest_fitness = min(pBest_fitness)

    for iteration in range(num_iterations):
        for i in range(num_particles):
            # Ažuriranje čestica
            particles[i], velocities[i] = update_particle(particles[i], velocities[i], pBest[i], gBest, w, c1, c2)
            current_fitness = fitness({i for i, bit in enumerate(particles[i]) if bit == 1 and i < len(node_list)}, G,shortest_paths)

            # Ažuriranje pBest
            if current_fitness < pBest_fitness[i]:
                pBest[i] = particles[i]
                pBest_fitness[i] = current_fitness

        # Ažuriranje gBest
        best_particle_index = pBest_fitness.index(min(pBest_fitness))
        if pBest_fitness[best_particle_index] < gBest_fitness:
            gBest = pBest[best_particle_index]
            gBest_fitness = pBest_fitness[best_particle_index]

        print(f"Iteracija {iteration+1}, Najbolji fitnes: {gBest_fitness}")


    optimal_set = {node_list[i] for i, bit in enumerate(gBest) if bit == 1 and i < len(node_list)}
    print("\nOptimalni rešavajući skup:", optimal_set)
    return optimal_set

In [9]:
# G = nx.petersen_graph()

# G = nx.Graph()
# G.add_edges_from([(1,2),(1,3),(1,5),(1,6),(2,4),(2,6),(3,5),(3,4),(4,5),(4,6),(5,6)])

G = load_gcol('grafovi\gcol\gcol14.txt')

n = len(G.nodes)

# Parametri PSO algoritma
num_particles = 500  # Broj čestica
num_iterations = 500  # Broj iteracija
w = 0.7  # Inercijalni faktor
c1 = 0.5  # Koeficijent ubrzanja ka pBest
c2 = 1.5 # Koeficijent ubrzanja ka gBest

shortest_paths = dict(nx.all_pairs_shortest_path_length(G)) 


rez = particle_swarm_optimization(G, num_particles, num_iterations, w, c1, c2,shortest_paths)
print(is_resolving_set(rez,G))

<>:6: SyntaxWarning: invalid escape sequence '\g'
<>:6: SyntaxWarning: invalid escape sequence '\g'
C:\Users\Korisnik\AppData\Local\Temp\ipykernel_176564\3200427051.py:6: SyntaxWarning: invalid escape sequence '\g'
  G = load_gcol('grafovi\gcol\gcol14.txt')


Iteracija 1, Najbolji fitnes: 35
Iteracija 2, Najbolji fitnes: 34
Iteracija 3, Najbolji fitnes: 28
Iteracija 4, Najbolji fitnes: 28
Iteracija 5, Najbolji fitnes: 27
Iteracija 6, Najbolji fitnes: 27
Iteracija 7, Najbolji fitnes: 25
Iteracija 8, Najbolji fitnes: 25
Iteracija 9, Najbolji fitnes: 25
Iteracija 10, Najbolji fitnes: 24
Iteracija 11, Najbolji fitnes: 24
Iteracija 12, Najbolji fitnes: 24
Iteracija 13, Najbolji fitnes: 23
Iteracija 14, Najbolji fitnes: 23
Iteracija 15, Najbolji fitnes: 23
Iteracija 16, Najbolji fitnes: 23
Iteracija 17, Najbolji fitnes: 22
Iteracija 18, Najbolji fitnes: 22
Iteracija 19, Najbolji fitnes: 22
Iteracija 20, Najbolji fitnes: 22
Iteracija 21, Najbolji fitnes: 22
Iteracija 22, Najbolji fitnes: 22
Iteracija 23, Najbolji fitnes: 22
Iteracija 24, Najbolji fitnes: 22
Iteracija 25, Najbolji fitnes: 22
Iteracija 26, Najbolji fitnes: 22
Iteracija 27, Najbolji fitnes: 22
Iteracija 28, Najbolji fitnes: 22
Iteracija 29, Najbolji fitnes: 22
Iteracija 30, Najbolji 

C:\Users\Korisnik\AppData\Local\Temp\ipykernel_176564\3200427051.py:6: SyntaxWarning: invalid escape sequence '\g'
  G = load_gcol('grafovi\gcol\gcol14.txt')


KeyboardInterrupt: 